In [1]:
# imports
import lal
import lalsimulation as lalsim
import numpy as np
import xphm_mrb as mrb
import matplotlib.pyplot as plt
import pickle

In [2]:
# event specific info
T_long = 128
T = 16
Fs = 1024
tgps = 1249852257.0
f_min = 20.
f_max = 512.
f_ref = 100.
detstrings = ['H1', 'L1', 'V1']

# # ========================================================================================
# # If data_GW190814.pkl exists in the path, then you may leave this section commented out 
# # in order to save time. Otherwise, run this to generate the data:
# data_GW190814 = mrb.data(T, Fs, T_long, tgps, detstrings = detstrings) 
# afile = open('data_GW190814.pkl', 'wb')
# pickle.dump(data_GW190814, afile)
# afile.close()
# # ========================================================================================

#reload object from file
file2 = open('data_GW190814.pkl', 'rb')
data = pickle.load(file2)
file2.close()

In [3]:
# some parameter dictionaries for use

example_pdict = {'mass_1': 27.72983831269138, 
 'mass_2': 2.50837808512176035,
 'chi_1x': -0.14718763312200078, 
 'chi_1y': -0.16835002087493475, 
 'chi_1z': 0.1272682221931026, 
 'chi_2x': 0.20356967254066038, 
 'chi_2y': -0.4010835740047465, 
 'chi_2z': -0.18169190464489784, 
 'distance': 1.0163824217045286e+25, 
 'thetaJN': 2.9019314196865764, 
 'kappa': -0.41903862210880943, 
 'ra': 2.6832891377862698, 
 'dec': 0.8551747550420822, 
 'psi': 1.4249910399143244, 
 'tc': 0.011820773134029385, 
 'tgps': tgps, 
 'tcoarse': T/2, 
 'f_min': f_min, 
 'f_max': f_max, 
 'f_ref': f_ref,
 'lal_dict': lal.CreateDict()}

fiducial_pdict = {'mass_1': 24.435483426456848, 
 'mass_2': 2.723374225187505,
 'chi_1x': 0.02359450525836797, 
 'chi_1y': 0.03868527217784936, 
 'chi_1z': 0.0024360175827902618, 
 'chi_2x': -0.30533509006002735, 
 'chi_2y': -0.5743866829395613, 
 'chi_2z': -0.06772756565943047, 
 'distance': 8.741147565419736e+24, 
 'thetaJN': 2.729726304256051, 
 'kappa': 2.729726304256051, 
 'ra': 0.4116550230717184, 
 'dec': -0.5739309671444994, 
 'psi': 2.9623503046019186, 
 'tc': 0.011534364898266381, 
 'tgps': tgps, 
 'tcoarse': T/2, 
 'f_min': f_min, 
 'f_max': f_max, 
 'f_ref': f_ref,
 'lal_dict': lal.CreateDict()}

test_pdict = {'mass_1': 22.48, 
 'mass_2': 2.869,
 'chi_1x': 0.02342113796878262, 
 'chi_1y': -0.14571362415791864, 
 'chi_1z': -0.1048829540948196, 
 'chi_2x': -0.4764822484116526, 
 'chi_2y': 0.052711018384293686, 
 'chi_2z': -0.07540798150587324, 
 'distance': 9.065577550589078e+24, 
 'thetaJN': 0.377519020780241, 
 'kappa': -0.23401127867213417, 
 'ra': 0.21938490119335974, 
 'dec': -0.45131110485446746, 
 'psi': 1.533445663292355, 
 'tc': 0.01144241616184194, 
 'tgps': tgps, 
 'tcoarse': T/2, 
 'f_min': f_min, 
 'f_max': f_max, 
 'f_ref': f_ref,
 'lal_dict': lal.CreateDict()}

In [4]:
# build the model objects for each set of parameters
example_model = mrb.model(T, Fs, example_pdict, detstrings=detstrings)
fiducial_model = mrb.model(T, Fs, fiducial_pdict, detstrings=detstrings)
test_model = mrb.model(T, Fs, test_pdict, detstrings=detstrings)

In [5]:
# compute and print exact likelihoods for each set of parameters
example_lnL_exact = mrb.compute_exact_log_likelihood(data, example_model)
print('exact log likelihood for example parameters: ', example_lnL_exact)
fiducial_lnL_exact = mrb.compute_exact_log_likelihood(data, fiducial_model)
print('exact log likelihood for fiducial parameters: ', fiducial_lnL_exact)
test_lnL_exact = mrb.compute_exact_log_likelihood(data, test_model)
print('exact log likelihood for test parameters: ', test_lnL_exact)

exact log likelihood for example parameters:  235.95506800949516
exact log likelihood for fiducial parameters:  256.4871783569316
exact log likelihood for test parameters:  250.1763218913472


In [6]:
%%time
# precompute bins and summary data for original (modeless) relative binning and the two mode-by-mode schemes
# takes about 1 min
modeless_binning_info = mrb.setup_binning(data, fiducial_model, modebymode = False)
binning_info_mrb_1 = mrb.setup_binning(data, fiducial_model, test_model=test_model, scheme = 1)
binning_info_mrb_2 = mrb.setup_binning(data, fiducial_model, test_model=test_model, scheme = 2)

/Users/nathanielleslie/Research/GW/modebymode-relative-binning/xphm_mrb.py:598: RuntimeWarning: invalid value encountered in true_divide
  r = (hhat/hhat0)[:,:,fbin_ind]
/Users/nathanielleslie/Research/GW/modebymode-relative-binning/xphm_mrb.py:646: RuntimeWarning: invalid value encountered in true_divide
  r = (h/h0)[:,:,fbin_ind]


CPU times: user 54.5 s, sys: 302 ms, total: 54.8 s
Wall time: 54.9 s


In [7]:
# compute log likelihoods for the example parameters using all of the relative binning schemes
lnL_modeless = mrb.compute_modeless_relative_binning_log_likelihood(modeless_binning_info, example_pdict)
print('log likelihood for example parameters using original relative binning: ', lnL_modeless)
lnL_mrb_1 = mrb.compute_relative_binning_log_likelihood(binning_info_mrb_1, example_pdict, scheme = 1)
print('log likelihood for example parameters using mode-by-mode relative binning, scheme 1: ', lnL_mrb_1)
lnL_mrb_2 = mrb.compute_relative_binning_log_likelihood(binning_info_mrb_2, example_pdict, scheme = 2)
print('log likelihood for example parameters using mode-by-mode relative binning, scheme 2: ', lnL_mrb_2)
print('exact log likelihood for example parameters: ', example_lnL_exact)

log likelihood for example parameters using original relative binning:  236.83986214288532
log likelihood for example parameters using mode-by-mode relative binning, scheme 1:  235.82034871216823
log likelihood for example parameters using mode-by-mode relative binning, scheme 2:  235.8243146205615
exact log likelihood for example parameters:  235.95506800949516
